In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import pandas as pd
import re
import seaborn as sns
import datetime
from datetime import timedelta
import numpy as np

In [68]:
kayak = ['LAS', 'LAX', 'CHI', 'SFO', 'ORL', 'DEN', 'DCA', 'SEA', 'BOS']
#airline = 'LAX'

# The size of each step in days
day_delta = datetime.timedelta(days=1)

start_date = datetime.date(2018, 12, 22)
end_date = start_date + 11*day_delta

url_list = []
for i in range((end_date - start_date).days):
    sd = start_date + i*day_delta
    for k in kayak: 
        url = 'https://www.kayak.com/flights/NYC-' + str(k) + '/' + str(sd) + '?sort=bestflight_a&fs=stops=0'
        url_list.append(url)
        
url_list

['https://www.kayak.com/flights/NYC-LAS/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-LAX/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-CHI/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-SFO/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-ORL/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-DEN/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-DCA/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-SEA/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-BOS/2018-12-22?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-LAS/2018-12-23?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-LAX/2018-12-23?sort=bestflight_a&fs=stops=0',
 'https://www.kayak.com/flights/NYC-CHI/2018-12-23?sort=bestflight_a&fs=stops=0',
 'https://www.ka

In [55]:
url = 'https://www.kayak.com'
r = requests.get(url)
layer1Cookies = r.cookies

url2 = url + '/s/horizon/flights/results/FlightSearchPoll'


referer = 'https://www.kayak.com/flights/NYC-LAX/2018-12-22?sort=bestflight_a&fs=stops=0'
headers = {
    'Host': 'www.kayak.com',
    'User-Agent': 'Chrome/63 (Macintosh; Intel Mac OS X 10.11; rv:57.0) Gecko/20100101 Firefox/57.0',
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Referer': referer,
    'Content-Type': 'application/x-www-form-urlencoded',
    'X-CSRF': 'kAqI1NgGh$DJnEUpiSDOWpdQXzlgAwG8EVOCd$gXO08-hpumC4oNpaOjz15GO_q9a5FdZPonpC2kF4CBYjEPh14',
    'X-RequestId': 'flights#frontdoor#Ag$s9g',
    'X-Requested-With': 'XMLHttpRequest',
    'Content-Length': '1094'
}

url3 = 'https://www.kayak.com/flights/NYC-LAX/2018-12-22?sort=bestflight_a&fs=stops=0'
params1 = {
    'searchId':'',
    'poll':'true',
    'pollNumber':'0',
    'applyFilters':'true',
    'filterState':'',
    'useViewStateFilterState':'false',
    'pageNumber':'1',
    'append':'false',
    'pollingId':'593601',  #interesting. explore further
    'requestReason':'POLL',
    'isSecondPhase':'false',
    'textAndPageLocations':'bottom,right',
    'displayAdPageLocations':'none',
    'existingAds':'false',
    'activeLeg':'-1',
    'view':'list',
    'renderPlusMinusThreeFlex':'false',
    'renderAirlineStopsMatrix':'false',
    'renderFlexHeader':'true',
    'tab':'flights',
    'pageOrigin':'F..FD..M0',
    'src':'',
    'searchingAgain':'',
    'c2s':'',
    'po':'',
    'personality':'',
    'provider':'',
    'isMulticity':'false',
    'flex_category':'exact',
    'oneway':'false',
    'nearby_origin':'false',
    'nearby_destination':'false',
    'countrySearch':'false',
    'travelers':'1',
    'adults':'1',
    'seniors':'0',
    'youth':'0',
    'child':'0',
    'seatInfant':'0',
    'lapInfant':'0',
    'cabin':'e',
    'cabinDisplayType':'Economy',
    'vertical':'flights',
    'url':url3,
    'id':'',
    'navigateToResults':'false',
    'ajaxts':'',
    'scriptsMetadata':'',
    'stylesMetadata':'',
}


result = requests.post(url2, headers = headers, data = params1, cookies = layer1Cookies)
text = result.json()

text = BeautifulSoup(text['content'], 'html.parser')

In [70]:
def getPricePrediction(url):
    url = "https://www.kayak.com/s/horizon/flights/results/FlightPricePredictionAction"
    response = requests.get(url)
    doc = html.fromstring(response.text)

    flight_url = url
    response_flight = requests.get(flight_url)

    s_id = re.compile(r'searchID=(\w+)')
    matches = s_id.finditer(str(text))
    for m in matches:
        searchId = m.group(1)

    token = re.compile(r"""\"formtoken\":\"(.+)\"\,""")
    matches = token.finditer(response_flight.text)
    for m in matches:
        formtoken = m.group(1)

    params = {
        'searchId': str(searchId),
        'formtoken': str(token)
    }
    
    next = requests.post(url, headers=headers, data=params)
    advice = next.json()
    print(advice)

In [ ]:
for url in url_list:
    getPricePrediction(url)
    print

{'infoText': "Prices predicted to rise <br> <span class='price'>$19</span> by <span class='date'>Dec 4</span>", 'cssClass': 'buy', 'advice': 'buy', 'moreInfoText': "Our data scientists think these are the best prices you'll see for the next seven days.\nLike weather forecasters, though, they can't be 100% certain. Our purchase advisory, shown here, is based on an analysis of current and past prices.", 'buy': True, 'actionableAmountMsg': 'Prices will likely increase $19', 'adviceText': 'Our Advice: Buy', 'alertsCta': 'Track price changes with our Price Alert emails.', 'actionableDateMsg': 'Within 7 days'}
{'infoText': "Prices predicted to rise <br> <span class='price'>$19</span> by <span class='date'>Dec 4</span>", 'cssClass': 'buy', 'advice': 'buy', 'moreInfoText': "Our data scientists think these are the best prices you'll see for the next seven days.\nLike weather forecasters, though, they can't be 100% certain. Our purchase advisory, shown here, is based on an analysis of current an

In [44]:
def get_prices_times(text):
    doc = html.fromstring(str(text))
    flights = doc.xpath('//div[@class="Base-Results-HorizonResult Flights-Results-FlightResultItem phoenix-rising sleek rp-contrast "]')
    #print(flights)

    result = []
    for f in flights:
        price_info = f.get('aria-label')
        time_info = f.xpath('.//div//div//div//div[@class="col-info result-column"]')[0]
        time = time_info.xpath('.//div//div//ol//li//div//div//div//div//span//span[@class="depart-time base-time"]')[0].text
        am_pm = time_info.xpath('.//div//div//ol//li//div//div//div//div//span//span[@class="time-meridiem meridiem"]')[0].text
        
        regex_price = re.compile(r'(\$)(\d*)',re.VERBOSE)
        matches_price = regex_price.finditer(price_info)
        for match in matches_price:
            clean_price = match.group(2)
            entry = {
                'price' : clean_price,
                'time' : time + am_pm
            }
            result.append(entry)
        
    return result

In [45]:
def get_flights(html):
    flights = html.find_all('li', 'flight')
    result = []
    for f in flights:
        info = str(f.find('div', 'bottom'))
        info = info.replace('<div class="bottom">', '')
        info = info.replace("</div>", '')
        result.append(info)
        
    return result

In [46]:
get_prices_times(text)

[{'price': '344', 'time': '10:40 pm'},
 {'price': '509', 'time': '6:00 pm'},
 {'price': '538', 'time': '9:15 pm'},
 {'price': '344', 'time': '9:40 pm'},
 {'price': '401', 'time': '11:52 am'},
 {'price': '413', 'time': '10:50 pm'},
 {'price': '413', 'time': '5:59 pm'},
 {'price': '423', 'time': '9:59 pm'},
 {'price': '426', 'time': '2:40 pm'},
 {'price': '430', 'time': '11:52 am'},
 {'price': '433', 'time': '8:35 am'},
 {'price': '447', 'time': '8:35 am'},
 {'price': '456', 'time': '4:49 pm'},
 {'price': '467', 'time': '7:00 am'},
 {'price': '476', 'time': '8:14 pm'},
 {'price': '482', 'time': '8:45 pm'},
 {'price': '488', 'time': '10:15 am'}]

In [47]:
get_flights(text)

['JetBlue',
 'United Airlines',
 'Delta',
 'JetBlue',
 'Spirit Airlines',
 'Frontier',
 'Frontier',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines',
 'Spirit Airlines']